In [2]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)


what we do is in the above cell we have some words each of embedding size 3
Then we send a query and compute the similarity of this query to every other word in torch.tensor

In [3]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x, query)
    
print(attn_scores_2)
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [4]:
query = inputs[1] #the second word i.e token is our query
context_vec_2 = torch.zeros(query.shape)
for i, x in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x
print(context_vec_2)
    
    


tensor([0.4419, 0.6515, 0.5683])


Now instead of computing attention score for one query we compute attention score with all queries
i.e we compute for each token its dot product with all other tokens to compute attention scores


In [5]:
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


Same task can be done with matrix multiplication

In [6]:
attn_scores = inputs @inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [7]:
attn_weights= torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


We will implement the self-attention mechanism step by step by introducing the
three trainable weight matrices Wq , Wk , and Wv . These three matrices are used to
project the embedded input tokens, x(i) , into query, key, and value vectors, respec-
tively

In [8]:
all_context_vecs= attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [9]:
x_2 = inputs[1] #the input token which in this case is the second word
d_in = inputs.shape[1]
d_out = 2

In [10]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
query_2 = x_2@W_query
key_2 = x_2@W_key
value_2=x_2@W_value
print(inputs.shape)
print(W_value.shape)

torch.Size([6, 3])
torch.Size([3, 2])


Even though in this case our goal is only one query and to find the attention of all other words wrt to second word we still have to find keys and value vectors of all other words

In [11]:
keys = inputs @W_key
values = inputs @W_value
print('keys.shape: ', keys.shape)
print('values.shape: ', values.shape)

keys.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


first lets compute similarity between second words query and second words key

In [12]:
keys_2 = keys[1]
attn_score_22= query_2.dot(keys_2)
#We can genealize this
attn_scores_2 = query_2 @keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [14]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2/ d_k**0.5, dim = -1)
print(attn_weights_2)
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
tensor([0.3061, 0.8210])


Now we consolidate all this functionality into one class

In [26]:
import torch.nn as nn
class SelfAttentionV1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query= nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key= nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        keys = x @self.W_key
        queries = x@self.W_query
        values= x@self.W_value
        attn_scores= queries @keys.T
        attn_weights = torch.softmax(attn_scores/ keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [27]:
torch.manual_seed(123)
sa_v1 = SelfAttentionV1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.3507, 0.8808],
        [0.3566, 0.8973],
        [0.3563, 0.8966],
        [0.3464, 0.8692],
        [0.3446, 0.8644],
        [0.3502, 0.8795]], grad_fn=<MmBackward0>)


Instead of using nn.Parameter we can use nn.Linear 

In [31]:
import torch.nn as nn
class SelfAttentionV2(nn.Module):
    def __init__(self, d_in, d_out, hasBias = False):
        super().__init__()
        self.W_query= nn.Linear(d_in, d_out, bias=hasBias)
        self.W_value = nn.Linear(d_in, d_out, bias=hasBias)
        self.W_key= nn.Linear(d_in, d_out, bias=hasBias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values= self.W_value(x)
        attn_scores= queries @keys.T
        attn_weights = torch.softmax(attn_scores/ keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [32]:
torch.manual_seed(789)
sa_v2 = SelfAttentionV2(d_in, d_out)
print(sa_v2(inputs))

tensor([[ 0.0293, -0.3242],
        [ 0.0284, -0.3250],
        [ 0.0284, -0.3251],
        [ 0.0271, -0.3272],
        [ 0.0267, -0.3278],
        [ 0.0277, -0.3262]], grad_fn=<MmBackward0>)


Now we will use masking to hide all the next tokens so that a token has only previous tokens visible and it cant see next token


In [33]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1667, 0.1635, 0.1641, 0.1672, 0.1767, 0.1618],
        [0.1675, 0.1622, 0.1626, 0.1688, 0.1746, 0.1642],
        [0.1675, 0.1623, 0.1627, 0.1687, 0.1745, 0.1643],
        [0.1672, 0.1642, 0.1644, 0.1680, 0.1705, 0.1657],
        [0.1672, 0.1646, 0.1648, 0.1678, 0.1694, 0.1661],
        [0.1673, 0.1634, 0.1637, 0.1682, 0.1723, 0.1650]],
       grad_fn=<SoftmaxBackward0>)


In [34]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


Now, we can multiply this mask with the attention weights to zero-out the values above
the diagonal:

In [36]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1675, 0.1622, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1675, 0.1623, 0.1627, 0.0000, 0.0000, 0.0000],
        [0.1672, 0.1642, 0.1644, 0.1680, 0.0000, 0.0000],
        [0.1672, 0.1646, 0.1648, 0.1678, 0.1694, 0.0000],
        [0.1673, 0.1634, 0.1637, 0.1682, 0.1723, 0.1650]],
       grad_fn=<MulBackward0>)


The third step is to renormalize the attention weights to sum up to 1 again in each
row. We can achieve this by dividing each element in each row by the sum in each row:

In [38]:
masked_simple_normalized = masked_simple / masked_simple.sum(dim=-1, keepdim=True)
print(masked_simple_normalized)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5080, 0.4920, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3401, 0.3295, 0.3303, 0.0000, 0.0000, 0.0000],
        [0.2519, 0.2474, 0.2477, 0.2530, 0.0000, 0.0000],
        [0.2005, 0.1975, 0.1976, 0.2012, 0.2032, 0.0000],
        [0.1673, 0.1634, 0.1637, 0.1682, 0.1723, 0.1650]],
       grad_fn=<DivBackward0>)


When we apply a mask and then renormalize the attention weights, it might initially
appear that information from future tokens (which we intend to mask) could still influ-
ence the current token because their values are part of the softmax calculation. How-
ever, the key insight is that when we renormalize the attention weights after masking,what we’re essentially doing is recalculating the softmax over a smaller subset (since
masked positions don’t contribute to the softmax value).
The mathematical elegance of softmax is that despite initially including all positions
in the denominator, after masking and renormalizing, the effect of the masked posi-
tions is nullified—they don’t contribute to the softmax score in any meaningful way.
In simpler terms, after masking and renormalization, the distribution of attention
weights is as if it was calculated only among the unmasked positions to begin with.
This ensures there’s no information leakage from future (or otherwise masked)
tokens as we intended.

Another way which is more efficient is that before taking the softmax of the attention weights we can make the upper diagonals as - inifinity
When we take the softmax it will automatically make the upper diagonal as 0 along with normalizing as well

In [39]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[-0.0634,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.0921, -0.1375,    -inf,    -inf,    -inf,    -inf],
        [-0.0908, -0.1357, -0.1322,    -inf,    -inf,    -inf],
        [-0.0514, -0.0774, -0.0757, -0.0452,    -inf,    -inf],
        [-0.0420, -0.0635, -0.0623, -0.0367, -0.0229,    -inf],
        [-0.0672, -0.1005, -0.0980, -0.0596, -0.0254, -0.0873]],
       grad_fn=<MaskedFillBackward0>)


In [40]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5080, 0.4920, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3401, 0.3295, 0.3303, 0.0000, 0.0000, 0.0000],
        [0.2519, 0.2474, 0.2477, 0.2530, 0.0000, 0.0000],
        [0.2005, 0.1975, 0.1976, 0.2012, 0.2032, 0.0000],
        [0.1673, 0.1634, 0.1637, 0.1682, 0.1723, 0.1650]],
       grad_fn=<SoftmaxBackward0>)
